퍼셉트론은 엄밀히 말하면 딥러닝은 아니지만 뉴런과 상당히 닮은 분류기이다.
뉴런의 계산과정을 이해하고 뉴런 하나만으로 AND, Or 연산과 같은 기초적인 연산을 할 수 잇음을 알아본다.

In [1]:
# 퍼셉트론 코드의 이해도를 높이기 위해 1은 True로 0은 False로 그리고 편향(bias)는 1로 지정한다.
T = 1.0
F = 0.0
bias = 1.0

In [6]:
# AND, OR, XOR 실습 데이터를 함수 형태로 정의해서 함수를 호출해서 데이터를 획득할 수 있게 한다.
def get_AND_data():
    X = [
        [F, F, bias],
        [F, T, bias],
        [T, F, bias],
        [F, F, bias]
    ] # 학습 데이터
    Y = [
        [F],
        [F],
        [F],
        [T]
    ] # 레이블, 클래스, 타겟, 실제값
    return X, Y
    
def get_OR_data():
    X = [[F, F, bias], [F, T, bias], [T, F, bias], [T, T, bias]]
    Y = [[F], [T], [T], [T]]
    return X, Y

def get_XOR_data():
    X = [[F, F, bias], [F, T, bias], [T, F, bias], [T, T, bias]]
    Y = [[F], [T], [T], [F]]
    return X, Y

In [8]:
# AND 연산에 필요한 데이터를 호출한다.
X, Y = get_AND_data()
X, Y

([[0.0, 0.0, 1.0], [0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [0.0, 0.0, 1.0]],
 [[0.0], [0.0], [0.0], [1.0]])

In [13]:
# tensorflow를 사용해서 퍼셉트론을 구현한다.
# 논리 연산을 위한 입력값으로 X, Y, 편향값을 입력받아(총 3개의 입력) 1개의 값을 출력해야 하므로 가중치는 [3, 1]로 설정한다.
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

W = tf.Variable(tf.random_normal([3, 1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W))

[[-1.6575176]
 [-1.1003543]
 [-0.0357269]]


퍼셉트론은 두 개의 입력이 있을 때 하나의 뉼너으로 두 개의 입력을 계산한 뒤 최종 결과값으로 0 또는 1을 출력한다.
두 개의 입력은 가중치와 곱해져서 뉴런의 첫 번째 단계인 시그마로 들어가고 시그마 단계에서 모든 가중치 * 입력값과 편향값을 더하는 과정이 이루어진다.
시그마 값은 뉴런의 두 번째 단계의 활성화 함수의 입력으로 들어가는데 퍼셉트론 스텝 함수라는 활성화 함수를 사용한다. 스텝 함수는 시그마 값 >= 0일 경우 1, 시그마 값 < 0일 경우 0을 출력한다.

In [15]:
# x가 0 이상이면 1, x가 0보다 작으면 0을 리턴하는 활성화 함수(스텝 함수)를 구현한다.
# tf.greater(a, b): a > b일 경우 True, 아니면 False
# tf.greater_equal(a, b): a >= b일 경우 True, 그렇지 않으면 False를 리턴한다.
# tf.less(a, b): a >= b일 경우 True, 그렇지 않으면 False를 리턴한다.
# tf.less_equal(a, b): a <= b일 경우 True, 그렇지 않으면 False를 리턴한다.
# tf.equal(a, b): a = b일 경우 True, 그렇지 않으면 False를 리턴한다.
# tf.not_equal(a, b): a <> b일 경우 True, 그렇지 않으면 False를 리턴한다.

sess = tf.Session()
f = tf.cast(tf.greater_equal(1, 0), dtype=tf.float32)
print(sess.run(f))

1.0


In [23]:
def step(x):
    return tf.cast(tf.greater_equal(x, 0), dtype=tf.float32)

In [26]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 손실(오차, 비용) 함수로 평균 제곱근 오차(MSE)를 사용한다.
f = tf.matmul(X, W) # 행렬 곱셈 함수로 데이터와 가중치를 곱한다.
output = step(f) # 스텝 함수를 실행한다. => 예측값
error = tf.subtract(Y, output) # 레이블과 스텝 함수의 실행 결과의 편차를 계산한다.
mse = tf.reduce_mean(tf.square(error))

보통의 경우 딥러닝 모델은 경사하강법을 통해 매개 변수를 최적화하지만 미분값이 큰 의미가 없는 스텝 함수의 영향으로(입력값이 0일 때 미분 불가능, 입력값이 1일 때 미분값은 항상 0) 퍼셉트론은 아래와 같은 방법으로 최적화를 진행한다.
***
결과값이 1이 나와야 하는데 스텝 함수의 출력이 0일 경우 => w_new = w_old + delta
결과값이 0이 나와야 하는데 스텝 함수의 출력이 1일 경우 => w_new = w_old - delta
위의 2가지 경우를 다음과 같이 사용할 수 있다.

In [27]:
import numpy as np
# 행렬의 transpose는 행과 열을 교환하여 얻는 전치행렬을 의미한다.
# matmul() 함수의 transpose_a = True 옵션을 사용하면 행렬을 곱하기 전에 1번째 행렬을 전치하고 transpose_b=True 옵션을 사용하면 행렬을 곱하기 전에 2번째 행렬을 전치해서 곱셈한다.
delta = tf.matmul(X, error, transpose_a=True)
# assign() 함수는 세션 안에서 텐서플로우 값을 변경한다.
train = tf.assign(W, tf.add(W, delta)) # 역전파

In [25]:
# 학습 후 테스트
sess = tf.Session()
sess.run(tf.global_variables_initializer())

err = 1
epoch = 0 # 학습 횟수
max_epochs = 20

# 에러가 발생하고 학습 횟수가 최대 학습 횟수를 넘어가지 않는 동안 반복한다.
while err > 0.0 and epoch < max_epochs:
    epoch += 1 # 학습 횟수를 증가시킨다.
    err = sess.run(mse) # 평균 제곱 오차를 계산한다.
    sess.run(train) # 학습시킨다.
    print('epoch: {}, mse: {}'.format(epoch, err))
# ==== while

print('테스트 결과\n', sess.run(output), sep='')
sess.close()

epoch: 1, mse: 0.75
epoch: 2, mse: 0.25
epoch: 3, mse: 0.25
epoch: 4, mse: 0.25
epoch: 5, mse: 0.25
epoch: 6, mse: 0.25
epoch: 7, mse: 0.25
epoch: 8, mse: 0.25
epoch: 9, mse: 0.25
epoch: 10, mse: 0.25
epoch: 11, mse: 0.25
epoch: 12, mse: 0.25
epoch: 13, mse: 0.25
epoch: 14, mse: 0.25
epoch: 15, mse: 0.25
epoch: 16, mse: 0.25
epoch: 17, mse: 0.25
epoch: 18, mse: 0.25
epoch: 19, mse: 0.25
epoch: 20, mse: 0.25
테스트 결과
[[1.]
 [0.]
 [0.]
 [1.]]


In [29]:
# 학습 후 테스트
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    err = 1
    epoch = 0 # 학습 횟수
    max_epochs = 20

    # 에러가 발생하고 학습 횟수가 최대 학습 횟수를 넘어가지 않는 동안 반복한다.
    while err > 0.0 and epoch < max_epochs:
        epoch += 1 # 학습 횟수를 증가시킨다.
        err = sess.run(mse) # 평균 제곱 오차를 계산한다.
        sess.run(train) # 학습시킨다.
        print('epoch: {}, mse: {}'.format(epoch, err))
    # ==== while

    print('테스트 결과\n', sess.run(output), sep='')

epoch: 1, mse: 0.75
epoch: 2, mse: 0.25
epoch: 3, mse: 0.25
epoch: 4, mse: 0.5
epoch: 5, mse: 0.25
epoch: 6, mse: 0.25
epoch: 7, mse: 0.25
epoch: 8, mse: 0.25
epoch: 9, mse: 0.25
epoch: 10, mse: 0.25
epoch: 11, mse: 0.25
epoch: 12, mse: 0.25
epoch: 13, mse: 0.25
epoch: 14, mse: 0.25
epoch: 15, mse: 0.25
epoch: 16, mse: 0.25
epoch: 17, mse: 0.25
epoch: 18, mse: 0.25
epoch: 19, mse: 0.25
epoch: 20, mse: 0.25
테스트 결과
[[1.]
 [0.]
 [0.]
 [1.]]
